In [1]:
# 自动重新加载修改后的模块
%load_ext autoreload
%autoreload 2

import os   
import gc
import torch
from IPython.display import clear_output
# 设置环境变量防止内存碎片
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
gc.collect() # 强制进行垃圾回收
torch.cuda.empty_cache() # 清空PyTorch的CUDA缓存
clear_output()  # 可选，清除输出避免混乱

# %tb
from steer.vector_generators.vector_generators import BaseVectorGenerator
from steer.vector_appliers.vector_applier import BaseVectorApplier
from steer.datasets import prepare_train_dataset, prepare_generation_datasets
import hydra, sys
from omegaconf import DictConfig

from hydra.core.global_hydra import GlobalHydra
GlobalHydra.instance().clear()
hydra.initialize(version_base='1.2', config_path='./hparams/Steer')
top_cfg = hydra.compose(config_name='Steer_config.yaml')

# @hydra.main(version_base='1.2',config_path='./hparams/Steer', config_name='Steer_config.yaml')
# def main(top_cfg: DictConfig):

print("Global Config:", top_cfg, "\n")
# Prepare datasets
# You can use the datasets defined in Steer_config.yaml or define your own here
train_datasets = {
    'reasoning':[
        {
            'question': '1 + 1 = ', 
            'matching':'</think>\n\n1 + 1 equals 2. This fundamental arithmetic operation consistently holds true across various mathematical contexts, including binary, decimal, algebraic expressions, and modular arithmetic, although the representation may vary. In standard arithmetic, the sum of two ones is always two.<｜end▁of▁sentence｜>', 
            'not_matching': "Alright, so I'm trying to figure out what 1 + 1 equals. Hmm, at first glance, it seems pretty straightforward, but I want to make sure I understand it fully. Let me think about how addition works. When you add two numbers, you're combining their quantities. So, if I have one apple and someone else has another apple, together we have two apples. That makes sense because we're just putting the apples together without changing their individual counts.\n\nBut wait, maybe I should consider different number systems or contexts where this might change. For example, in binary, which is the base-2 system, 1 + 1 equals 10. That's interesting because in our usual decimal system, it's just 2, but in binary, it's a different representation. So, the way we add numbers can vary depending on the base we're using.\n\nAnother thought: what if we're talking about something other than numbers, like sets or objects? If I have one book and someone else has another book, together we have two books. It's the same concept, just adding the quantities. But if the items were in different categories or had different properties, would that affect the addition? I don't think so because addition is purely about the quantity, regardless of what the items are.\n\nI also wonder about the history of addition. How did humans figure out that combining two quantities gives a sum? It must have been through counting and recognizing patterns. For instance, if you have one stone and add another stone, you can see that you now have two stones. This simple concept likely formed the basis of mathematical addition.\n\nWhat about in mathematics, specifically in algebra? If I have variables, say x + x, that simplifies to 2x. So, in that case, 1 + 1 would be 2. It's consistent with the basic arithmetic we learned earlier. But what if it's more complex, like adding fractions or decimals? For example, 1/2 + 1/2 equals 1, and 0.5 + 0.5 also equals 1. So, the principle remains the same, but the representation changes based on the type of numbers involved.\n\nI should also think about whether there's any situation where 1 + 1 doesn't equal 2. In standard mathematics, across all number systems, 1 + 1 equals 2. Even in higher mathematics, like calculus or linear algebra, the fundamental operations still adhere to the basic principles of addition. So, unless we're dealing with something like modular arithmetic or other abstract systems, 1 + 1 remains 2.\n\nWait, in modular arithmetic, 1 + 1 modulo 2 would be 0. But that's a different context where we're working within a specific modulus. So, it's still 2 in the usual sense, but modulo 2, it's 0. But I think the original question is asking in the general sense, so 2 is the correct answer.\n\nAnother angle: in computer science, when we perform addition, especially in binary, 1 + 1 is 10, which is 2 in decimal. So, the result is the same, just represented differently. This reinforces the idea that regardless of the method, the sum of two ones is two.\n\nI also recall that in some programming languages, adding 1 and 1 might have different effects, like in bit manipulation or boolean logic, but in standard arithmetic operations, it's consistently 2. So, unless specified otherwise, 1 + 1 equals 2.\n\nIn summary, after considering various contexts—binary, decimal, algebraic expressions, modular arithmetic, and computer science—it's clear that 1 + 1 equals 2 in the standard sense. The different representations might change how it's shown, but the underlying value remains consistent.\n</think>\n\n1 + 1 equals 2. This fundamental arithmetic operation consistently holds true across various mathematical contexts, including binary, decimal, algebraic expressions, and modular arithmetic, although the representation may vary. In standard arithmetic, the sum of two ones is always two.<｜end▁of▁sentence｜>"
        }
    ]
}
# train_datasets = prepare_train_dataset(top_cfg)
generation_datasets={
    'reasoning':[
        {'input': "1 + 1 = ,"} # , "9.8 or 9.11, which is bigger?"
    ]
}
# generation_datasets = prepare_generation_datasets(top_cfg)

# Generate Steering Vectors
vector_generator = BaseVectorGenerator(top_cfg)
vectors = vector_generator.generate_vectors(train_datasets)
print(vectors)


# Apply Vectors to Model 
vector_applier = BaseVectorApplier(top_cfg)
for dataset in vectors.keys():
    print(f"Applying  {dataset} vectors to model ...")
    vector_applier.apply_vectors(vectors[dataset])

# vector_applier.apply_vectors()

# Result Generation
vector_applier.generate(generation_datasets)

# Resets the model to its initial state, clearing any modifications.
vector_applier.model.reset_all()

# if __name__=='__main__':
#     main(top_cfg)

We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


Global Config: {'model_name_or_path': '/home/ubuntu/models/QwQ-32B-AWQ', 'torch_dtype': 'float32', 'device': 'cuda:0', 'seed': 0, 'use_chat_template': False, 'system_prompt': '', 'steer_train_hparam_paths': ['hparams/Steer/lm_steer_hparams/generate_lm_steer.yaml'], 'steer_train_dataset': 'toxicity', 'save_vectors': True, 'steer_vector_output_dir': 'steer/vectors/Qwen3-0.6B/', 'apply_steer_hparam_paths': ['hparams/Steer/lm_steer_hparams/apply_lm_steer.yaml'], 'steer_vector_load_dir': ['steer/vectors/QwQ-32B-AWQ/reasoning/lm_steer_vector'], 'generation_data': ['nontoxic'], 'generation_data_size': 100, 'generation_output_dir': 'steer/logs/QwQ-32B-AWQ/', 'num_responses': 1, 'steer_from_end_position': False, 'generation_params': {'max_new_tokens': 1024, 'temperature': 0.9, 'do_sample': True}} 

LM_STEER Generator Hyperparameters:
LmSteerHyperParams(use_chat_template=False, system_prompt='', torch_dtype='float32', seed=0, model_name_or_path='/home/ubuntu/models/QwQ-32B-AWQ', device='cuda:0',

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

number of training steps: 1000


  0%|          | 0/1000 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB. GPU 0 has a total capacty of 23.55 GiB of which 30.56 MiB is free. Including non-PyTorch memory, this process has 23.38 GiB memory in use. Of the allocated memory 22.81 GiB is allocated by PyTorch, and 126.31 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [26]:
vectors['reasoning']['lm_steer']['projector2']
# ipykernel_launcher.py: error: unrecognized arguments: --f=/run/user/1000/jupyter/runtime/kernel-v3eb0fd67609e1c26eccfe72369719a6ef6e5bdb46.json

Parameter containing:
tensor([[[ 0.0422, -0.0858,  0.0970,  ...,  0.0206, -0.0167, -0.0496],
         [ 0.1246, -0.1356,  0.1253,  ...,  0.1437,  0.1677, -0.1462],
         [-0.0160,  0.0279, -0.0272,  ..., -0.0337, -0.0124,  0.0360],
         ...,
         [ 0.0266,  0.0026, -0.0388,  ..., -0.0086,  0.0086,  0.0733],
         [-0.0621,  0.0482, -0.0679,  ..., -0.0238, -0.0712,  0.0588],
         [ 0.0359, -0.0533,  0.0486,  ...,  0.0653,  0.0330, -0.0589]],

        [[ 0.0486, -0.0329, -0.1395,  ...,  0.2526,  0.0511, -0.0493],
         [-0.0917,  0.1143, -0.4360,  ...,  0.1194, -0.0986,  0.0967],
         [ 0.1363, -0.1332,  0.2460,  ..., -0.0332,  0.1025, -0.0984],
         ...,
         [ 0.1195, -0.1198, -0.3671,  ...,  0.0778,  0.1165, -0.1509],
         [ 0.0316, -0.0550,  0.2766,  ..., -0.2376, -0.0084, -0.0044],
         [-0.0773,  0.0495,  0.1641,  ...,  0.0592, -0.0790,  0.0708]]],
       requires_grad=True)